In [1]:
!pip install --upgrade scikit-learn
!pip install --upgrade keras
!pip install --upgrade tensorflow
!pip install --upgrade pandas
!pip install --upgrade numpy
!pip install --upgrade matplotlib
!pip install --upgrade torch

  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.14.0 requires keras<2.15,>=2.14.0, but you have keras 2.15.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.8/208.8 MB 17.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 40.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninst

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()


# download the data from the given URLs and store them in dataframes
sample_submission_df = pd.read_csv('sample_submission.csv')
training_data_df = pd.read_csv('training_data.csv')
unlabelled_test_data_df = pd.read_csv('unlabelled_test_data.csv')

# Exemple de données
texts = training_data_df['sentence'].values.tolist()
labels = training_data_df['difficulty'].values.tolist()



# Création de la matrice TF-IDF
vectorizer = TfidfVectorizer(max_features=14585)  # Ajustez ce nombre pour qu'il corresponde à la taille de vos données
X = vectorizer.fit_transform(texts).toarray()
y = np.array(labels)
y = label_encoder.fit_transform(labels)  # 'labels' est votre liste ou tableau d'étiquettes

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
# Conversion en tenseurs PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Création de DataLoaders
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32  # Vous pouvez ajuster la taille du batch
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [54]:
# Neural Network Class
class TextClassifier(nn.Module):
    def __init__(self, input_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 128)
        # Adjust the number of output classes if necessary
        self.fc3 = nn.Linear(128, len(np.unique(y)))  

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

model = TextClassifier(14585)  # 5000 correspond au nombre de max_features dans TF-IDF

In [55]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 500  # Ajustez le nombre d'époques selon vos besoins
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item()}")

Epoch 1/500 - Loss: 1.7822530269622803
Epoch 2/500 - Loss: 1.7875423431396484
Epoch 3/500 - Loss: 1.767717957496643
Epoch 4/500 - Loss: 1.6979035139083862
Epoch 5/500 - Loss: 1.5938019752502441
Epoch 6/500 - Loss: 1.494519591331482
Epoch 7/500 - Loss: 1.4121994972229004
Epoch 8/500 - Loss: 1.2029496431350708
Epoch 9/500 - Loss: 1.1869138479232788
Epoch 10/500 - Loss: 1.026456356048584
Epoch 11/500 - Loss: 0.8689619898796082
Epoch 12/500 - Loss: 0.7471699714660645
Epoch 13/500 - Loss: 0.6695020198822021
Epoch 14/500 - Loss: 0.5264548063278198
Epoch 15/500 - Loss: 0.6328009366989136
Epoch 16/500 - Loss: 0.47326719760894775
Epoch 17/500 - Loss: 0.4435458481311798
Epoch 18/500 - Loss: 0.4558165371417999
Epoch 19/500 - Loss: 0.43707147240638733
Epoch 20/500 - Loss: 0.41791442036628723
Epoch 21/500 - Loss: 0.18402275443077087
Epoch 22/500 - Loss: 0.22072120010852814
Epoch 23/500 - Loss: 0.2759205996990204
Epoch 24/500 - Loss: 0.2714945375919342
Epoch 25/500 - Loss: 0.1926714926958084
Epoch 2

In [56]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Accuracy: 41.875%


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()


# download the data from the given URLs and store them in dataframes
sample_submission_df = pd.read_csv('sample_submission.csv')
training_data_df = pd.read_csv('training_data.csv')
unlabelled_test_data_df = pd.read_csv('unlabelled_test_data.csv')

# Exemple de données
texts = training_data_df['sentence'].values.tolist()
labels = training_data_df['difficulty'].values.tolist()



# Création de la matrice TF-IDF
vectorizer = TfidfVectorizer(max_features=14585)  # Ajustez ce nombre pour qu'il corresponde à la taille de vos données
X = vectorizer.fit_transform(texts).toarray()
y = np.array(labels)
y = label_encoder.fit_transform(labels)  # 'labels' est votre liste ou tableau d'étiquettes

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Conversion en tenseurs PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Création de DataLoaders
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32  # Vous pouvez ajuster la taille du batch
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)


class TextClassifier(nn.Module):
    def __init__(self, input_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        # Adjust the number of output classes if necessary
        self.fc3 = nn.Linear(64, len(np.unique(y)))  

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = nn.functional.softmax(x, dim=1)
        return x

model = TextClassifier(14585)  # 5000 correspond au nombre de max_features dans TF-IDF



criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 50  # Ajustez le nombre d'époques selon vos besoins
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item()}")



correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Epoch 1/50 - Loss: 1.789111852645874
Epoch 2/50 - Loss: 1.7874727249145508
Epoch 3/50 - Loss: 1.776065707206726
Epoch 4/50 - Loss: 1.7614822387695312
Epoch 5/50 - Loss: 1.720894455909729
Epoch 6/50 - Loss: 1.6484582424163818
Epoch 7/50 - Loss: 1.6783955097198486
Epoch 8/50 - Loss: 1.6745131015777588
Epoch 9/50 - Loss: 1.589257836341858
Epoch 10/50 - Loss: 1.5663212537765503
Epoch 11/50 - Loss: 1.4827903509140015
Epoch 12/50 - Loss: 1.4737046957015991
Epoch 13/50 - Loss: 1.5106581449508667
Epoch 14/50 - Loss: 1.4452215433120728
Epoch 15/50 - Loss: 1.3828837871551514
Epoch 16/50 - Loss: 1.3381227254867554
Epoch 17/50 - Loss: 1.2659095525741577
Epoch 18/50 - Loss: 1.2498377561569214
Epoch 19/50 - Loss: 1.2201223373413086
Epoch 20/50 - Loss: 1.1883372068405151
Epoch 21/50 - Loss: 1.1768336296081543
Epoch 22/50 - Loss: 1.201076626777649
Epoch 23/50 - Loss: 1.1596307754516602
Epoch 24/50 - Loss: 1.1452516317367554
Epoch 25/50 - Loss: 1.160419225692749
Epoch 26/50 - Loss: 1.1300677061080933
E

In [41]:
from sklearn.model_selection import train_test_split, GridSearchCV
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# download the data from the given URLs and store them in dataframes
sample_submission_df = pd.read_csv('sample_submission.csv')
training_data_df = pd.read_csv('training_data.csv')
unlabelled_test_data_df = pd.read_csv('unlabelled_test_data.csv')

# Exemple de données
texts = training_data_df['sentence'].values.tolist()
labels = training_data_df['difficulty'].values.tolist()

# Création de la matrice TF-IDF
vectorizer = TfidfVectorizer(max_features=14585)  # Ajustez ce nombre pour qu'il corresponde à la taille de vos données
X = vectorizer.fit_transform(texts).toarray()
y = np.array(labels)
y = label_encoder.fit_transform(labels)  # 'labels' est votre liste ou tableau d'étiquettes

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Conversion en tenseurs PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Création de DataLoaders
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32  # Vous pouvez ajuster la taille du batch
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

# Neural Network Class
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Define the hyperparameters to search over
param_grid = {
    'hidden_dim1': [64, 128, 256],
    'hidden_dim2': [32, 64, 128],
    'lr': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64]
}

# Function to train and evaluate a model
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs=50):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
    
    # Evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Custom loop for hyperparameter tuning
best_accuracy = 0
best_params = {}

for hidden_dim1 in param_grid['hidden_dim1']:
    for hidden_dim2 in param_grid['hidden_dim2']:
        for lr in param_grid['lr']:
            for batch_size in param_grid['batch_size']:
                # Create the model
                model = TextClassifier(input_dim=X_train.shape[1], hidden_dim1=hidden_dim1, 
                                       hidden_dim2=hidden_dim2, output_dim=len(np.unique(y)))
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Create DataLoaders
                train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
                test_loader = DataLoader(test_data, batch_size=batch_size)

                # Train and evaluate the model
                accuracy = train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer)

                # Update best parameters if current model is better
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {'hidden_dim1': hidden_dim1, 'hidden_dim2': hidden_dim2, 'lr': lr, 'batch_size': batch_size}

print(f"Best hyperparameters: {best_params}")
print(f"Best Accuracy: {best_accuracy}%")

KeyboardInterrupt: 

Best practice to find the best parameters yet but too much time.


In [36]:
from sklearn.model_selection import train_test_split, GridSearchCV
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed

label_encoder = LabelEncoder()

# download the data from the given URLs and store them in dataframes
sample_submission_df = pd.read_csv('sample_submission.csv')
training_data_df = pd.read_csv('training_data.csv')
unlabelled_test_data_df = pd.read_csv('unlabelled_test_data.csv')

# Exemple de données
texts = training_data_df['sentence'].values.tolist()
labels = training_data_df['difficulty'].values.tolist()

# Création de la matrice TF-IDF
vectorizer = TfidfVectorizer(max_features=14585)  # Ajustez ce nombre pour qu'il corresponde à la taille de vos données
X = vectorizer.fit_transform(texts).toarray()
y = np.array(labels)
y = label_encoder.fit_transform(labels)  # 'labels' est votre liste ou tableau d'étiquettes

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Conversion en tenseurs PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Création de DataLoaders
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32  # Vous pouvez ajuster la taille du batch
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

# Neural Network Class
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Define the hyperparameters to search over
param_grid = {
    'hidden_dim1': [64, 128, 256],
    'hidden_dim2': [32, 64, 128],
    'lr': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64]
}

# Function to train and evaluate a model
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs=50):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
    
    # Evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Custom loop for hyperparameter tuning
best_accuracy = 0
best_params = {}

for hidden_dim1 in param_grid['hidden_dim1']:
    for hidden_dim2 in param_grid['hidden_dim2']:
        for lr in param_grid['lr']:
            for batch_size in param_grid['batch_size']:
                # Create the model
                model = TextClassifier(input_dim=X_train.shape[1], hidden_dim1=hidden_dim1, 
                                       hidden_dim2=hidden_dim2, output_dim=len(np.unique(y)))
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Create DataLoaders
                train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
                test_loader = DataLoader(test_data, batch_size=batch_size)

                # Train and evaluate the model
                accuracy = train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer)

                # Update best parameters if current model is better
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {'hidden_dim1': hidden_dim1, 'hidden_dim2': hidden_dim2, 'lr': lr, 'batch_size': batch_size}

print(f"Best hyperparameters: {best_params}")

# Custom loop for hyperparameter tuning
best_accuracy = 0
best_params = {}

def train_and_evaluate_model_parallel(model, train_loader, test_loader, criterion, optimizer, epochs=50):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
    
    # Evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

def hyperparameter_tuning(hidden_dim1, hidden_dim2, lr, batch_size):
    # Create the model
    model = TextClassifier(input_dim=X_train.shape[1], hidden_dim1=hidden_dim1, 
                           hidden_dim2=hidden_dim2, output_dim=len(np.unique(y)))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Create DataLoaders
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    # Train and evaluate the model
    accuracy = train_and_evaluate_model_parallel(model, train_loader, test_loader, criterion, optimizer)

    # Update best parameters if current model is better
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {'hidden_dim1': hidden_dim1, 'hidden_dim2': hidden_dim2, 'lr': lr, 'batch_size': batch_size}

Parallel(n_jobs=-1)(delayed(hyperparameter_tuning)(hidden_dim1, hidden_dim2, lr, batch_size) for hidden_dim1 in param_grid['hidden_dim1'] for hidden_dim2 in param_grid['hidden_dim2'] for lr in param_grid['lr'] for batch_size in param_grid['batch_size'])

print(f"Best hyperparameters: {best_params}")
print(f"Best Accuracy: {best_accuracy}%")

[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

In [37]:
from sklearn.model_selection import train_test_split, GridSearchCV
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed

label_encoder = LabelEncoder()

# download the data from the given URLs and store them in dataframes
sample_submission_df = pd.read_csv('sample_submission.csv')
training_data_df = pd.read_csv('training_data.csv')
unlabelled_test_data_df = pd.read_csv('unlabelled_test_data.csv')

# Exemple de données
texts = training_data_df['sentence'].values.tolist()
labels = training_data_df['difficulty'].values.tolist()

# Création de la matrice TF-IDF
vectorizer = TfidfVectorizer(max_features=14585)  # Ajustez ce nombre pour qu'il corresponde à la taille de vos données
X = vectorizer.fit_transform(texts).toarray()
y = np.array(labels)
y = label_encoder.fit_transform(labels)  # 'labels' est votre liste ou tableau d'étiquettes

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Conversion en tenseurs PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Création de DataLoaders
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32  # Vous pouvez ajuster la taille du batch
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

# Neural Network Class
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Define the hyperparameters to search over
param_grid = {
    'hidden_dim1': [64, 128, 256],
    'hidden_dim2': [32, 64, 128],
    'lr': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64]
}

# Function to train and evaluate a model
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
    
    # Evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Custom loop for hyperparameter tuning
best_accuracy = 0
best_params = {}

for hidden_dim1 in param_grid['hidden_dim1']:
    for hidden_dim2 in param_grid['hidden_dim2']:
        for lr in param_grid['lr']:
            for batch_size in param_grid['batch_size']:
                # Create the model
                model = TextClassifier(input_dim=X_train.shape[1], hidden_dim1=hidden_dim1, 
                                       hidden_dim2=hidden_dim2, output_dim=len(np.unique(y)))
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Create DataLoaders
                train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
                test_loader = DataLoader(test_data, batch_size=batch_size)

                # Train and evaluate the model
                accuracy = train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer)

                # Update best parameters if current model is better
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {'hidden_dim1': hidden_dim1, 'hidden_dim2': hidden_dim2, 'lr': lr, 'batch_size': batch_size}

print(f"Best hyperparameters: {best_params}")

# Custom loop for hyperparameter tuning
best_accuracy = 0
best_params = {}

def train_and_evaluate_model_parallel(model, train_loader, test_loader, criterion, optimizer, epochs=50):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
    
    # Evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

def hyperparameter_tuning(hidden_dim1, hidden_dim2, lr, batch_size):
    # Create the model
    model = TextClassifier(input_dim=X_train.shape[1], hidden_dim1=hidden_dim1, 
                           hidden_dim2=hidden_dim2, output_dim=len(np.unique(y)))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Create DataLoaders
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    # Train and evaluate the model
    accuracy = train_and_evaluate_model_parallel(model, train_loader, test_loader, criterion, optimizer)

    return accuracy, {'hidden_dim1': hidden_dim1, 'hidden_dim2': hidden_dim2, 'lr': lr, 'batch_size': batch_size}

# Run hyperparameter tuning in parallel
results = Parallel(n_jobs=-1)(delayed(hyperparameter_tuning)(hidden_dim1, hidden_dim2, lr, batch_size) 
                              for hidden_dim1 in param_grid['hidden_dim1'] 
                              for hidden_dim2 in param_grid['hidden_dim2'] 
                              for lr in param_grid['lr'] 
                              for batch_size in param_grid['batch_size'])

# Find the best parameters
best_accuracy = 0
best_params = None
for accuracy, params in results:
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

print(f"Best hyperparameters: {best_params}")
print(f"Best Accuracy: {best_accuracy}%")

Best hyperparameters: {'hidden_dim1': 64, 'hidden_dim2': 128, 'lr': 0.01, 'batch_size': 64}


KeyboardInterrupt: 

In [40]:
import torch
from torch.utils.data import DataLoader
from joblib import Parallel, delayed
import numpy as np

label_encoder = LabelEncoder()

# download the data from the given URLs and store them in dataframes
sample_submission_df = pd.read_csv('sample_submission.csv')
training_data_df = pd.read_csv('training_data.csv')
unlabelled_test_data_df = pd.read_csv('unlabelled_test_data.csv')

# Exemple de données
texts = training_data_df['sentence'].values.tolist()
labels = training_data_df['difficulty'].values.tolist()

# Création de la matrice TF-IDF
vectorizer = TfidfVectorizer(max_features=14585)  # Ajustez ce nombre pour qu'il corresponde à la taille de vos données
X = vectorizer.fit_transform(texts).toarray()
y = np.array(labels)
y = label_encoder.fit_transform(labels)  # 'labels' est votre liste ou tableau d'étiquettes

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Conversion en tenseurs PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Création de DataLoaders
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32  # Vous pouvez ajuster la taille du batch
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

# Neural Network Class
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Define the hyperparameters to search over
param_grid = {
    'hidden_dim1': [64, 128, 256],
    'hidden_dim2': [32, 64, 128],
    'lr': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64]
}

# Function to train and evaluate a model
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs=50):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
    
    import torch.nn as nn
    import torch.optim as optim

    # Define the TextClassifier model
    class TextClassifier(nn.Module):
        def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
            super(TextClassifier, self).__init__()
            self.fc1 = nn.Linear(input_dim, hidden_dim1)
            self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
            self.fc3 = nn.Linear(hidden_dim2, output_dim)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(p=0.5)

        def forward(self, x):
            x = self.fc1(x)
            x = self.relu(x)
            x = self.dropout(x)
            x = self.fc2(x)
            x = self.relu(x)
            x = self.dropout(x)
            x = self.fc3(x)
            return x

    # Load the data
    X_train = np.load('X_train.npy')
    X_test = np.load('X_test.npy')
    y_train = np.load('y_train.npy')
    y_test = np.load('y_test.npy')

    # Convert the data to PyTorch tensors
    X_train = torch.from_numpy(X_train).float()
    X_test = torch.from_numpy(X_test).float()
    y_train = torch.from_numpy(y_train).long()
    y_test = torch.from_numpy(y_test).long()

    # Define the hyperparameters
    hidden_dim1 = 64
    hidden_dim2 = 128
    lr = 0.01
    batch_size = 64

    # Create the model
    model = TextClassifier(input_dim=X_train.shape[1], hidden_dim1=hidden_dim1, 
                           hidden_dim2=hidden_dim2, output_dim=len(np.unique(y_train)))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Create DataLoaders
    train_data = torch.utils.data.TensorDataset(X_train, y_train)
    test_data = torch.utils.data.TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    # Train the model
    for epoch in range(50):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

    # Evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy}%")